<a href="https://colab.research.google.com/github/rexian/ML/blob/main/ollama/ollama_mistral_chromadb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install virtualenv
!mkdir -p deepeval
!cd deepeval
!virtualenv venv

created virtual environment CPython3.11.12.final.0-64 in 643ms
  creator CPython3Posix(dest=/content/venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


Install DeepEval

In [16]:
!pip3 install -U -q deepeval
!pip3 install -U -q chromadb
!pip3 install -U -q aiobotocore
!pip3 install -U -q aiogram openai python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.4/680.4 kB 11.4 MB/s eta 0:00:00


In [17]:
!deepeval login --confident-api-key API_KEY


🎉🥳 Congratulations! You've successfully logged in! 🙌 
You're now using DeepEval with Confident AI. Follow our quickstart tutorial 
here: ]8;id=392692;https://documentation.confident-ai.com/getting-started/installation\https://documentation.confident-ai.com/getting-started/installation]8;;\


## Setting Up the Environment
### Installing and serving Ollama
#### - Launching Xterm
%xterm
#### - Installing Ollama
curl https://ollama.ai/install.sh | sh
#### - Serving ollama
ollama serve &

In [18]:
!pip3 install colab-xterm

In [ ]:
%reload_ext colabxterm
%xterm

In [ ]:
!ollama pull mistral

In [22]:
!ollama list

NAME              ID              SIZE      MODIFIED      
mistral:latest    f974a74358d6    4.1 GB    8 seconds ago    


In [23]:
!pip3 install -U -q langchain-ollama

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="mistral")

chain = prompt | model

chain.invoke({"question": "What is hallucination in AI?"})

" In the context of AI, a hallucination does not have the same meaning as it does in human psychology. Instead, when we talk about hallucinations in AI, we often refer to situations where an AI interprets or generates data that is not actually present in the input. This could be due to the AI's programming or learning from biased or incorrect data. It's important to note that this is usually undesirable as it can lead to incorrect predictions or decisions by the AI system. To minimize hallucinations, AI models are typically trained on large and diverse datasets and regularly evaluated for accuracy and consistency."

In [25]:
import chromadb

# Initialize Chroma client
client = chromadb.PersistentClient(path="./chroma_db")

# Create or load a collection
collection = client.get_or_create_collection(name="rag_documents")

In [26]:
# Load an embedding model
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Example document chunks
document_chunks = [
    "Chroma is an open-source vector database for efficient embedding retrieval.",
    "It enables fast semantic search using vector similarity.",
    "Chroma retrieves relevant data with cosine similarity.",
]

# Store chunks with embeddings in Chroma
for i, chunk in enumerate(document_chunks):
    embedding = embedding_model.encode(chunk).tolist()  # Convert text to vector
    collection.add(
        ids=[str(i)],  # Unique ID for each document
        embeddings=[embedding],  # Vector representation
        metadatas=[{"text": chunk}]  # Store original text as metadata
    )

In [27]:
def search(query):
  query_embedding = embedding_model.encode(query).tolist()

  res = collection.query(
    query_embeddings=[query_embedding],
    n_results=3  # Retrieve top-K matches
  )
  result = []
  for val in res["metadatas"][0]:
    result.append(val['text'])
  return  result if result else None

query = "How does Chroma work?"
retrieval_context = search(query)
print(retrieval_context)

prompt_template = """
Answer the user question based on the supporting context.

User Question:
{input}

Supporting Context:
{retrieval_context}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

model = OllamaLLM(model="mistral")

chain = prompt | model

actual_output = chain.invoke({"input": query, "retrieval_context": retrieval_context})
print(actual_output)

#expected_output = "Chroma works by retrieving data that is semantically similar to a given input using a method called cosine similarity. This approach allows it to find and return the most relevant information from its indexed dataset based on the cosine angle between the vector representation of the input query and the vectors of the documents in the dataset."



['Chroma retrieves relevant data with cosine similarity.', 'Chroma is an open-source vector database for efficient embedding retrieval.', 'It enables fast semantic search using vector similarity.']
 Chroma works as an open-source vector database designed for efficient embedding retrieval. It uses a technique called cosine similarity to find and retrieve relevant data that are semantically similar to the query. This allows for fast and accurate semantic search based on vector similarity.
